# Load Libraries

In [55]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from rotation_forest import random_feature_subsets
from rotation_forest import RotationForestClassifier
from sklearn.ensemble._forest import ForestClassifier, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from lib.multi_adaboost import multi_AdaBoost
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [3]:
f_database = pd.read_csv("1_database/arrhythmia.data", header=None)
f_database

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1
448,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10
449,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2
450,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1


In [4]:
s_database = pd.read_csv("2_database/AnalizaHRVRhythmDataset.csv", delimiter=";", decimal=",")
s_database

,Spatial_filling_index_lag_1,Spatial_filling_index_lag_2,Spatial_filling_index_lag_3,Standard_deviation_ratio,Cardiac_sympathetic_index,Correlation_dimension_D2_lag_1,Correlation_dimension_D2_lag_2,Correlation_dimension_D2_lag_3,Central_tendency_measure_lag_1,Central_tendency_measure_lag_2,...,Recurrence_DET_lag_1,Recurrence_DET_lag_2,Recurrence_DET_lag_3,Recurrence_ShEn_lag_1,Recurrence_ShEn_lag_2,Recurrence_ShEn_lag_3,Recurrence_Lam_lag_1,Recurrence_Lam_lag_2,Recurrence_Lam_lag_3,signal_type
0,2.485403,0.000000,0.0,1.594752,0.627057,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,PAC
1,5.332129,0.000000,0.0,0.654815,1.527150,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal
2,5.381935,0.000000,0.0,0.863627,1.157908,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal
3,5.283447,0.000000,0.0,0.700930,1.426676,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal
4,5.342012,0.000000,0.0,0.585449,1.708092,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,3.644247,5.187902,0.0,2.572795,0.388682,0,0,0,0.0,0.0,...,0.327869,0.0,0.0,1.609438,0.0,0.0,0.0,0.0,0.0,PVC
8839,5.809989,5.490799,0.0,0.509318,1.963409,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal
8840,3.315160,5.237696,0.0,2.618265,0.381932,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,PVC
8841,5.312135,5.872565,0.0,1.448715,0.690267,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Normal


In [5]:
t_database = pd.read_csv("3_database/AnalizaCHFDataset.csv", delimiter=";", decimal=",")
t_database

,SFI_lag_1,SFI_lag_2,SFI_lag_3,SD1_SD2,CSI,CVI,CD2_lag_1,CD2_lag_2,CD2_lag_3,CTM_lag_1,...,DET_lag_1,DET_lag_2,DET_lag_3,ShEn_lag_1,ShEn_lag_2,ShEn_lag_3,Lam_lag_1,Lam_lag_2,Lam_lag_3,signal_type
0,2.865769,2.791715,2.757770,1.398731,0.714934,-1.880022,0.000467,0.000119,0.000115,0.881868,...,0.365718,0.123798,0.143330,1.968922,2.070400,0.564834,0.124937,0.083487,0.084323,ARIT
1,3.115207,3.116688,3.004405,0.775340,1.289757,-1.886136,0.000113,0.000028,0.000040,0.892670,...,0.340610,0.073182,0.091653,1.638407,1.611772,0.201114,0.162608,0.087626,0.082379,ARIT
2,4.634587,4.569919,4.574763,1.124426,0.889343,-1.711505,0.000801,0.000303,0.000219,0.740642,...,0.368471,0.113948,0.128036,1.780927,1.926669,0.576331,0.191263,0.137866,0.152324,ARIT
3,2.919805,2.867303,2.833440,1.346321,0.742765,-1.795332,0.000604,0.000127,0.000130,0.751366,...,0.384258,0.110500,0.138249,1.914978,2.093161,0.606183,0.115065,0.111160,0.134079,ARIT
4,2.483123,2.398590,2.351488,1.608056,0.621869,-1.626827,0.002544,0.000763,0.000522,0.814917,...,0.412246,0.228928,0.237267,2.170245,2.518404,0.960252,0.271704,0.264403,0.220745,ARIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,8.067921,7.846301,7.699194,1.138031,0.878710,-1.517783,0.026332,0.011665,0.013823,0.783333,...,0.473618,0.458117,0.334045,3.022183,3.422413,3.253907,0.388651,0.356639,0.309509,CHF
3313,9.295793,9.064647,8.868989,1.418917,0.704763,-1.945013,0.003371,0.001521,0.001478,0.930526,...,0.427870,0.396942,0.352812,3.647965,3.622037,2.908596,0.315759,0.290246,0.269686,CHF
3314,8.143015,7.834063,7.534534,1.523050,0.656577,-2.067162,0.000661,0.000189,0.000262,0.908722,...,0.446077,0.393662,0.347431,3.427065,3.419593,2.408961,0.409376,0.377535,0.355430,CHF
3315,9.204318,9.024964,8.833244,1.645147,0.607849,-2.034452,0.000918,0.000314,0.000347,0.918280,...,0.447366,0.403165,0.398920,3.456781,3.648036,2.072851,0.402229,0.371165,0.368865,CHF


# Check data

### Dataset 1

In [6]:
f_database.info()
f_database.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Columns: 280 entries, 0 to 279
dtypes: float64(120), int64(155), object(5)
memory usage: 988.9+ KB


0      0
1      0
2      0
3      0
4      0
      ..
275    0
276    0
277    0
278    0
279    0
Length: 280, dtype: int64

In [7]:
objects = f_database.select_dtypes(include=['object'])
objects

,10,11,12,13,14
0,13,64,-2,?,63
1,37,-17,31,?,53
2,34,70,66,23,75
3,11,-5,20,?,71
4,13,61,3,?,?
...,...,...,...,...,...
447,4,40,-27,?,63
448,66,52,79,?,73
449,-19,-61,-70,84,84
450,29,-22,43,103,80


In [8]:
objects.where(objects == "?").count()

10      8
11     22
12      1
13    376
14      1
dtype: int64

In [9]:
f_database = f_database[f_database[12] != '?']
f_database = f_database[f_database[14] != '?']

f_database[12] = pd.to_numeric(f_database[12])
f_database[14] = pd.to_numeric(f_database[12])

f_database[f_database[10] == '?'] = np.nan
f_database[10] = pd.to_numeric(f_database[10])
f_database[f_database[10].isna()] = f_database.loc[:,10].mean()

f_database[f_database[11] == '?'] = np.nan
f_database[11] = pd.to_numeric(f_database[11])
f_database[f_database[11].isna()] = f_database.loc[:,11].mean()

f_database.drop(columns=[13], inplace=True)
f_database

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8.0
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6.0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10.0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1.0
5,13.0,0.0,169.0,51.0,100.0,167.0,321.0,174.0,91.0,107.0,...,-0.6,12.2,-2.8,0.0,0.0,0.9,2.2,13.5,31.1,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53.0,1.0,160.0,70.0,80.0,199.0,382.0,154.0,117.0,-37.0,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1.0
448,37.0,0.0,190.0,85.0,100.0,137.0,361.0,201.0,73.0,86.0,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10.0
449,36.0,0.0,166.0,68.0,108.0,176.0,365.0,194.0,116.0,-85.0,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2.0
450,32.0,1.0,155.0,55.0,93.0,106.0,386.0,218.0,63.0,54.0,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1.0


In [10]:
f_database.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 451
Columns: 279 entries, 0 to 279
dtypes: float64(279)
memory usage: 984.4 KB


### Dataset 2

In [13]:
s_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8843 entries, 0 to 8842
Columns: 231 entries, Spatial_filling_index_lag_1 to signal_type
dtypes: float64(172), int64(31), object(28)
memory usage: 15.6+ MB


In [32]:
null_values_counter = s_database.isnull().sum()
null_values_counter.where(na_values_counter > 0).dropna()

Series([], dtype: float64)

In [14]:
objects = s_database.select_dtypes(include=['object'])
objects

,A_exists,B_exists,C_exists,D_exists,E_exists,F_exists,G_exists,H_exists,I_exists,J_exists,...,S_exists,T_exists,U_exists,V_exists,W_exists,X_exists,Y_exists,Z_exists,AA_exists,signal_type
0,yes,yes,yes,yes,no,no,no,no,yes,yes,...,no,no,no,no,no,no,no,yes,no,PAC
1,yes,yes,no,yes,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,no,no,Normal
2,yes,yes,no,yes,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,no,no,Normal
3,yes,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,Normal
4,yes,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,yes,no,yes,no,no,no,no,no,yes,no,...,no,no,no,yes,no,no,no,yes,no,PVC
8839,yes,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,Normal
8840,yes,no,yes,no,no,no,no,no,yes,no,...,no,no,no,yes,no,no,no,yes,no,PVC
8841,yes,no,yes,no,no,no,no,no,yes,yes,...,yes,no,no,no,no,no,no,no,no,Normal


In [15]:
objects.nunique()

A_exists       2
B_exists       2
C_exists       2
D_exists       2
E_exists       2
F_exists       2
G_exists       2
H_exists       2
I_exists       2
J_exists       2
K_exists       2
L_exists       2
M_exists       2
N_exists       2
O_exists       2
P_exists       2
Q_exists       2
R_exists       2
S_exists       2
T_exists       2
U_exists       2
V_exists       2
W_exists       2
X_exists       2
Y_exists       2
Z_exists       2
AA_exists      2
signal_type    9
dtype: int64

In [17]:
columns_to_replace = objects.nunique().index
s_database[columns_to_replace] = s_database[columns_to_replace].replace(('yes', 'no'), (1, 0))
s_database[columns_to_replace]

,A_exists,B_exists,C_exists,D_exists,E_exists,F_exists,G_exists,H_exists,I_exists,J_exists,...,S_exists,T_exists,U_exists,V_exists,W_exists,X_exists,Y_exists,Z_exists,AA_exists,signal_type
0,1,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,PAC
1,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Normal
2,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Normal
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normal
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,PVC
8839,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normal
8840,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,PVC
8841,1,0,1,0,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,Normal


In [20]:
le = LabelEncoder()
le.fit(s_database['signal_type'])
s_database['signal_type'] = le.transform(s_database['signal_type'])
s_database

,Spatial_filling_index_lag_1,Spatial_filling_index_lag_2,Spatial_filling_index_lag_3,Standard_deviation_ratio,Cardiac_sympathetic_index,Correlation_dimension_D2_lag_1,Correlation_dimension_D2_lag_2,Correlation_dimension_D2_lag_3,Central_tendency_measure_lag_1,Central_tendency_measure_lag_2,...,Recurrence_DET_lag_1,Recurrence_DET_lag_2,Recurrence_DET_lag_3,Recurrence_ShEn_lag_1,Recurrence_ShEn_lag_2,Recurrence_ShEn_lag_3,Recurrence_Lam_lag_1,Recurrence_Lam_lag_2,Recurrence_Lam_lag_3,signal_type
0,2.485403,0.000000,0.0,1.594752,0.627057,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,4
1,5.332129,0.000000,0.0,0.654815,1.527150,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3
2,5.381935,0.000000,0.0,0.863627,1.157908,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3
3,5.283447,0.000000,0.0,0.700930,1.426676,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3
4,5.342012,0.000000,0.0,0.585449,1.708092,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,3.644247,5.187902,0.0,2.572795,0.388682,0,0,0,0.0,0.0,...,0.327869,0.0,0.0,1.609438,0.0,0.0,0.0,0.0,0.0,5
8839,5.809989,5.490799,0.0,0.509318,1.963409,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3
8840,3.315160,5.237696,0.0,2.618265,0.381932,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,5
8841,5.312135,5.872565,0.0,1.448715,0.690267,0,0,0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,3


In [22]:
s_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8843 entries, 0 to 8842
Columns: 231 entries, Spatial_filling_index_lag_1 to signal_type
dtypes: float64(172), int32(1), int64(58)
memory usage: 15.6 MB


### Dataset 3

In [24]:
t_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3317 entries, 0 to 3316
Columns: 238 entries, SFI_lag_1  to  signal_type
dtypes: float64(193), int64(17), object(28)
memory usage: 6.0+ MB


In [33]:
null_values_counter = t_database.isnull().sum()
null_values_counter.where(na_values_counter > 0).dropna()

Series([], dtype: float64)

In [34]:
objects = t_database.select_dtypes(include=['object'])
objects

,A_exists,B_exists,C_exists,D_exists,G_exists,E_exists,I_exists,F_exists,H_exists,J_exists,...,V_exists,N_exists,O_exists,Q_exists,R_exists,AA_exists,Z_exists,X_exists,W_exists,signal_type
0,yes,yes,yes,yes,no,no,no,no,yes,yes,...,no,no,no,no,no,no,no,yes,no,ARIT
1,yes,yes,yes,yes,yes,no,no,no,yes,yes,...,no,no,no,no,no,no,no,yes,no,ARIT
2,yes,yes,yes,yes,yes,no,no,no,yes,yes,...,no,no,no,no,no,no,no,yes,no,ARIT
3,yes,yes,yes,yes,yes,no,no,no,yes,yes,...,no,no,no,no,yes,no,yes,yes,no,ARIT
4,yes,yes,yes,yes,yes,no,no,no,yes,yes,...,no,no,no,no,no,no,no,yes,no,ARIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,yes,no,yes,yes,yes,no,yes,no,yes,yes,...,no,no,no,no,no,no,yes,yes,no,CHF
3313,yes,no,yes,no,no,no,no,no,yes,no,...,no,no,no,no,no,no,no,yes,no,CHF
3314,yes,no,yes,no,no,no,no,no,yes,no,...,no,no,no,no,no,no,no,yes,no,CHF
3315,yes,no,yes,no,no,no,no,no,yes,no,...,no,no,no,no,no,no,no,yes,no,CHF


In [37]:
columns_to_replace = objects.nunique().index
t_database[columns_to_replace] = t_database[columns_to_replace].replace(('yes', 'no'), (1, 0))
t_database[columns_to_replace]

,A_exists,B_exists,C_exists,D_exists,G_exists,E_exists,I_exists,F_exists,H_exists,J_exists,...,V_exists,N_exists,O_exists,Q_exists,R_exists,AA_exists,Z_exists,X_exists,W_exists,signal_type
0,1,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,ARIT
1,1,1,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,ARIT
2,1,1,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,ARIT
3,1,1,1,1,1,0,0,0,1,1,...,0,0,0,0,1,0,1,1,0,ARIT
4,1,1,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,ARIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,1,0,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,1,1,0,CHF
3313,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,CHF
3314,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,CHF
3315,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,CHF


In [39]:
le = LabelEncoder()
le.fit(t_database[' signal_type'])
t_database[' signal_type'] = le.transform(t_database[' signal_type'])
t_database

,SFI_lag_1,SFI_lag_2,SFI_lag_3,SD1_SD2,CSI,CVI,CD2_lag_1,CD2_lag_2,CD2_lag_3,CTM_lag_1,...,DET_lag_1,DET_lag_2,DET_lag_3,ShEn_lag_1,ShEn_lag_2,ShEn_lag_3,Lam_lag_1,Lam_lag_2,Lam_lag_3,signal_type
0,2.865769,2.791715,2.757770,1.398731,0.714934,-1.880022,0.000467,0.000119,0.000115,0.881868,...,0.365718,0.123798,0.143330,1.968922,2.070400,0.564834,0.124937,0.083487,0.084323,0
1,3.115207,3.116688,3.004405,0.775340,1.289757,-1.886136,0.000113,0.000028,0.000040,0.892670,...,0.340610,0.073182,0.091653,1.638407,1.611772,0.201114,0.162608,0.087626,0.082379,0
2,4.634587,4.569919,4.574763,1.124426,0.889343,-1.711505,0.000801,0.000303,0.000219,0.740642,...,0.368471,0.113948,0.128036,1.780927,1.926669,0.576331,0.191263,0.137866,0.152324,0
3,2.919805,2.867303,2.833440,1.346321,0.742765,-1.795332,0.000604,0.000127,0.000130,0.751366,...,0.384258,0.110500,0.138249,1.914978,2.093161,0.606183,0.115065,0.111160,0.134079,0
4,2.483123,2.398590,2.351488,1.608056,0.621869,-1.626827,0.002544,0.000763,0.000522,0.814917,...,0.412246,0.228928,0.237267,2.170245,2.518404,0.960252,0.271704,0.264403,0.220745,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,8.067921,7.846301,7.699194,1.138031,0.878710,-1.517783,0.026332,0.011665,0.013823,0.783333,...,0.473618,0.458117,0.334045,3.022183,3.422413,3.253907,0.388651,0.356639,0.309509,1
3313,9.295793,9.064647,8.868989,1.418917,0.704763,-1.945013,0.003371,0.001521,0.001478,0.930526,...,0.427870,0.396942,0.352812,3.647965,3.622037,2.908596,0.315759,0.290246,0.269686,1
3314,8.143015,7.834063,7.534534,1.523050,0.656577,-2.067162,0.000661,0.000189,0.000262,0.908722,...,0.446077,0.393662,0.347431,3.427065,3.419593,2.408961,0.409376,0.377535,0.355430,1
3315,9.204318,9.024964,8.833244,1.645147,0.607849,-2.034452,0.000918,0.000314,0.000347,0.918280,...,0.447366,0.403165,0.398920,3.456781,3.648036,2.072851,0.402229,0.371165,0.368865,1


In [40]:
t_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3317 entries, 0 to 3316
Columns: 238 entries, SFI_lag_1  to  signal_type
dtypes: float64(193), int32(1), int64(44)
memory usage: 6.0 MB


# Seperate target column

In [44]:
y_f = f_database[279].apply(np.int32)
X_f = f_database.drop([279], axis = 1)
print(f'X_f : {X_f.shape}')
print(f'y_f : {y_f.shape}')

X_f : (450, 278)
y_f : (450,)


In [45]:
y_s = s_database['signal_type'].apply(np.int32)
X_s = s_database.drop(['signal_type'], axis = 1)
print(f'X_s : {X_s.shape}')
print(f'y_s : {y_s.shape}')

X_s : (8843, 230)
y_s : (8843,)


In [46]:
y_t = t_database[' signal_type'].apply(np.int32)
X_t = t_database.drop([' signal_type'], axis = 1)
print(f'X_t : {X_t.shape}')
print(f'y_t : {y_t.shape}')

X_t : (3317, 237)
y_t : (3317,)


# Random Forest

### Method

In [84]:
def random_forest_classification(dataset, y, X, details):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if(details):
        print(f'X_train : {X_train.shape}')
        print(f'X_test : {X_test.shape}')
        print(f'y_train : {y_train.shape}')
        print(f'y_test : {y_test.shape} \n')
        
    start = time.time()
    
    rf_Model = RandomForestClassifier(n_estimators = 100, max_depth=20)
    rf_Model.fit(X_train, y_train)
    
    scores = cross_val_score(rf_Model, X, y, cv=10)

    end = time.time()
    print("rotation_forest_classification")
    print('Accuracy: ',  round(scores.mean(), 5), ' +- ',  round(scores.std(), 5),)
    print(f"Runtime of the program is ", round(end - start, 2), "s", ' \n')

### Dataset 1

In [85]:
random_forest_classification(f_database, y_f, X_f, True)

X_train : (360, 278)
X_test : (90, 278)
y_train : (360,)
y_test : (90,) 

rotation_forest_classification
Accuracy:  0.76255  +-  0.04501
Runtime of the program is  7.66 s  



### Dataset 2

In [50]:
random_forest_classification(s_database, y_s, X_s, True)

X_train : (7074, 230)
X_test : (1769, 230)
y_train : (7074,)
y_test : (1769,) 

Accuracy:  0.7812710029314729  +-  0.06629009138024497


### Dataset 3

In [ ]:
random_forest_classification(t_database, y_t, X_t, True)

# Rotation Forest

### Method

In [90]:
def rotation_forest_classification(dataset, y, X, details):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if(details):
        print(f'X_train : {X_train.shape}')
        print(f'X_test : {X_test.shape}')
        print(f'y_train : {y_train.shape}')
        print(f'y_test : {y_test.shape} \n')
        
    start = time.time()
    
    rtf_Model = RotationForestClassifier(n_estimators = 30, max_depth=20)
    rtf_Model.fit(X_train, y_train)

    scores = cross_val_score(rtf_Model, X, y, cv=10)
    
    end = time.time()
    print("rotation_forest_classification")
    print('Accuracy: ',  round(scores.mean(), 5), ' +- ',  round(scores.std(), 5),)
    print(f"Runtime of the program is ", round(end - start, 2), "s", ' \n')

### Dataset 1

In [91]:
rotation_forest_classification(f_database, y_f, X_f, True)

X_train : (360, 278)
X_test : (90, 278)
y_train : (360,)
y_test : (90,) 

rotation_forest_classification
Accuracy:  0.64664  +-  0.07756
Runtime of the program is  122.77 s  



### Dataset 2

In [ ]:
rotation_forest_classification(s_database, y_s, X_s, True)

### Dataset 3

In [ ]:
rotation_forest_classification(t_database, y_t, X_t, True)

# AdaBoost.M1

### Method

In [92]:
def adaboost_m1_classification(dataset, y, X, details):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if(details):
        print(f'X_train : {X_train.shape}')
        print(f'X_test : {X_test.shape}')
        print(f'y_train : {y_train.shape}')
        print(f'y_test : {y_test.shape} \n')
        
    start = time.time()
    
    adaB_Model = AdaBoostClassifier(n_estimators = 40)
    adaB_Model.fit(X_train, y_train)
    
    scores = cross_val_score(adaB_Model, X, y, cv=10)
    
    end = time.time()
    print("adaboost_m1_classification")
    print('Accuracy: ',  round(scores.mean(), 5), ' +- ',  round(scores.std(), 5),)
    print(f"Runtime of the program is ", round(end - start, 2), "s", ' \n')

### Dataset 1

In [93]:
adaboost_m1_classification(f_database, y_f, X_f, True)

X_train : (360, 278)
X_test : (90, 278)
y_train : (360,)
y_test : (90,) 

adaboost_m1_classification
Accuracy:  0.64694  +-  0.03464
Runtime of the program is  5.2 s  



### Dataset 2

In [94]:
adaboost_m1_classification(s_database, y_s, X_s, True)

X_train : (7074, 230)
X_test : (1769, 230)
y_train : (7074,)
y_test : (1769,) 

adaboost_m1_classification
Accuracy:  0.47643  +-  0.14146
Runtime of the program is  66.36 s  



### Dataset 3

In [ ]:
adaboost_m1_classification(t_database, y_t, X_t, True)

# MultiBoost

### Method

In [96]:
def multi_adaboost_m1_classification(dataset, y, X, details):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if(details):
        print(f'X_train : {X_train.shape}')
        print(f'X_test : {X_test.shape}')
        print(f'y_train : {y_train.shape}')
        print(f'y_test : {y_test.shape} \n')
        
    start = time.time()
    
    mad_Model = multi_AdaBoost.AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=40)
    mad_Model.fit(X_train, y_train)
    
    
    print("multi_adaboost_m1_classification")
    print('Accuracy: ',accuracy_score(mad_Model.predict(X_test),y_test))
    end = time.time()
    print(f"Runtime of the program is ", round(end - start, 2), "s", ' \n')

### Dataset 1

In [97]:
multi_adaboost_m1_classification(f_database, y_f, X_f, True)

X_train : (360, 278)
X_test : (90, 278)
y_train : (360,)
y_test : (90,) 

multi_adaboost_m1_classification
Accuracy:  0.6555555555555556
Runtime of the program is  0.98 s  



### Dataset 2

In [98]:
multi_adaboost_m1_classification(s_database, y_s, X_s, True)

X_train : (7074, 230)
X_test : (1769, 230)
y_train : (7074,)
y_test : (1769,) 

multi_adaboost_m1_classification
Accuracy:  0.5166760881854154
Runtime of the program is  5.94 s  



### Dataset 3

In [99]:
multi_adaboost_m1_classification(t_database, y_t, X_t, True)

X_train : (2653, 237)
X_test : (664, 237)
y_train : (2653,)
y_test : (664,) 

multi_adaboost_m1_classification
Accuracy:  0.8298192771084337
Runtime of the program is  4.25 s  



# Support Vector Machines

### Method

In [82]:
def svm_method(dataset, y, X, details):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if(details):
        print(f'X_train : {X_train.shape}')
        print(f'X_test : {X_test.shape}')
        print(f'y_train : {y_train.shape}')
        print(f'y_test : {y_test.shape} \n')
    
    types = ["rbf", "linear", "poly"]
    for element in types:
        start = time.time()
        if element == "linear":
            clf = svm.SVC(kernel=element, C=1)
        elif element == "poly":
            clf = svm.SVC(kernel=element, C=0.03)
        else:
            clf = svm.SVC(kernel=element, C=5, gamma=0.15)
        clf.fit(X_train, y_train)

        scores = cross_val_score(clf, X, y, cv=10)
        
        end = time.time()
        print("svm_method")
        print('Type: ', element)  
        print('Accuracy: ',  round(scores.mean(), 5), ' +- ',  round(scores.std(), 5),)
        print(f"Runtime of the program is ", round(end - start, 2), "s", ' \n')

### Dataset 1

In [83]:
svm_method(f_database, y_f, X_f, True)

X_train : (360, 278)
X_test : (90, 278)
y_train : (360,)
y_test : (90,) 

svm_method
Type:  rbf
Accuracy:  0.59387  +-  0.0304
Runtime of the program is  3.25 s  

svm_method
Type:  linear
Accuracy:  0.66719  +-  0.08063
Runtime of the program is  0.98 s  

svm_method
Type:  poly
Accuracy:  0.57569  +-  0.02224
Runtime of the program is  0.58 s  



### Dataset 2

In [100]:
svm_method(s_database, y_s, X_s, True)

X_train : (7074, 230)
X_test : (1769, 230)
y_train : (7074,)
y_test : (1769,) 



KeyboardInterrupt: 

### Dataset 3

In [ ]:
svm_method(t_database, y_t, X_t, True)

X_train : (2653, 237)
X_test : (664, 237)
y_train : (2653,)
y_test : (664,) 

svm_method
Type:  rbf
Accuracy:  0.40036  +-  0.00278
Runtime of the program is  70.65 s  

